In [1]:
import gymnasium as gym
env = gym.make("CartPole-v1")

In [4]:
env.action_space

Discrete(2)

In [5]:
env.reset()

(array([-0.03861868,  0.00684651,  0.04572421, -0.00346667], dtype=float32),
 {})

In [ ]:
env.step(env.action_space.sample())

In [ ]:
# returns an initial observation
env.reset()
for i in range(20):
   # env.action_space.sample() produces either 0 (left) or 1 (right).
   observation,reward,done,info =env.step(env.action_space.sample())
   print("step", i, observation, reward, done, info)
env.close()

For Google Colab: Colab does not have a video mode

In [ ]:
# install dependencies needed for recording videos
!apt-get install -y xvfb x11-utils
!pip install pyvirtualdisplay==0.2.*

In [ ]:
from pyvirtualdisplay import Display
display = Display(visible=False, size=(1400, 900))
_ = display.start()

In [ ]:
from gym.wrappers.monitoring.video_recorder import VideoRecorder
before_training = "before_training.mp4"
video = VideoRecorder(env, before_training)
# returns an initial observation
env.reset()
for i in range(200):
   env.render()
   video.capture_frame()
   # env.action_space.sample() produces either 0 (left) or 1 (right).
   observation, reward, done, info = env.step(env.action_space.sample())
   # Not printing this time
   # print("step", i, observation, reward, done, info)
video.close()
env.close()

In [ ]:
from base64 import b64encode
def render_mp4(videopath: str) -> str:
  """
  Gets a string containing a b4-encoded version of the MP4 video
  at the specified path.
  """
  mp4 = open(videopath, 'rb').read()
  base64_encoded_mp4 = b64encode(mp4).decode()
  return f'<video width=400 controls><source src="data:video/mp4;' \
         f'base64,{base64_encoded_mp4}" type="video/mp4"></video>'

In [ ]:
from IPython.display import HTML
html = render_mp4(before_training)
HTML(html)

In [ ]:
import ray
from ray.rllib.agents.ppo import PPOTrainer
config = {
   "env": "CartPole-v1",
   
   # Change the following line to `“framework”: “tf”` to use tensorflow
   # "framework": "torch",
   "framework": "tf2",
   "model": {
      "fcnet_hiddens": [32],
      "fcnet_activation": "linear",
   },
}
stop = {"episode_reward_mean": 195}
ray.shutdown()
ray.init(
   num_cpus=3,
   include_dashboard=False,
   ignore_reinit_error=True,
   log_to_driver=False,
)
# execute training
analysis = ray.tune.run(
   "PPO",
   config=config,
   stop=stop,
   checkpoint_at_end=True,
)

## poetry install tensorflow_probability ? 
